In [1]:
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, TimeSeriesSplit

In [3]:
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
f = pd.read_csv('C:\\ny_engineered.csv')
df = pd.DataFrame(f)

In [5]:
df['date'] = pd.to_datetime(df.date)

In [6]:
# Setting DateTime index
df['date'] = pd.to_datetime(df.date)
df.set_index('date', inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39455 entries, 2015-07-02 00:00:00 to 2019-12-31 22:00:00
Data columns (total 45 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   load         39455 non-null  float64
 1   temp         39455 non-null  float64
 2   humid        39455 non-null  float64
 3   target       39455 non-null  float64
 4   cwkend       39455 non-null  float64
 5   mth_sin      39455 non-null  float64
 6   mth_cos      39455 non-null  float64
 7   hour_sin     39455 non-null  float64
 8   hour_cos     39455 non-null  float64
 9   hour_num     39455 non-null  float64
 10  mth_num      39455 non-null  float64
 11  cmonth_2.0   39455 non-null  int64  
 12  cmonth_3.0   39455 non-null  int64  
 13  cmonth_4.0   39455 non-null  int64  
 14  cmonth_5.0   39455 non-null  int64  
 15  cmonth_6.0   39455 non-null  int64  
 16  cmonth_7.0   39455 non-null  int64  
 17  cmonth_8.0   39455 non-null  int64  
 18  cmonth_9.0 

In [8]:
# Change the integer variables to int32 type to reduce memory usage
df[['load', 'humid', 'target', 'hour_num', 'mth_num']] = \
    df[['load', 'humid', 'target', 'hour_num', 'mth_num']].astype(np.int32)

In [9]:
# Change the dummy variables to uint8 type to reduce memory usage
df[['cmonth_2.0', 'cmonth_3.0', 'cmonth_4.0', 'cmonth_5.0', 'cmonth_6.0', 'cmonth_7.0', 'cmonth_8.0', 'cmonth_9.0',
    'cmonth_10.0', 'cmonth_11.0', 'cmonth_12.0', 'chour_1.0', 'chour_2.0', 'chour_3.0', 'chour_4.0', 'chour_5.0',
    'chour_6.0', 'chour_7.0', 'chour_8.0', 'chour_9.0', 'chour_10.0', 'chour_11.0', 'chour_12.0', 'chour_13.0', 
    'chour_14.0', 'chour_15.0', 'chour_16.0', 'chour_17.0', 'chour_18.0', 'chour_19.0', 'chour_20.0', 'chour_21.0',
    'chour_22.0', 'chour_23.0', 'cwkend']] = \
    df[['cmonth_2.0', 'cmonth_3.0', 'cmonth_4.0', 'cmonth_5.0', 'cmonth_6.0', 'cmonth_7.0', 'cmonth_8.0', 
        'cmonth_9.0', 'cmonth_10.0', 'cmonth_11.0', 'cmonth_12.0', 'chour_1.0', 'chour_2.0', 'chour_3.0',
        'chour_4.0', 'chour_5.0', 'chour_6.0', 'chour_7.0', 'chour_8.0', 'chour_9.0', 'chour_10.0', 'chour_11.0',
        'chour_12.0', 'chour_13.0', 'chour_14.0', 'chour_15.0', 'chour_16.0', 'chour_17.0', 'chour_18.0',
        'chour_19.0', 'chour_20.0', 'chour_21.0', 'chour_22.0', 'chour_23.0', 'cwkend']].astype(np.uint8)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39455 entries, 2015-07-02 00:00:00 to 2019-12-31 22:00:00
Data columns (total 45 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   load         39455 non-null  int32  
 1   temp         39455 non-null  float64
 2   humid        39455 non-null  int32  
 3   target       39455 non-null  int32  
 4   cwkend       39455 non-null  uint8  
 5   mth_sin      39455 non-null  float64
 6   mth_cos      39455 non-null  float64
 7   hour_sin     39455 non-null  float64
 8   hour_cos     39455 non-null  float64
 9   hour_num     39455 non-null  int32  
 10  mth_num      39455 non-null  int32  
 11  cmonth_2.0   39455 non-null  uint8  
 12  cmonth_3.0   39455 non-null  uint8  
 13  cmonth_4.0   39455 non-null  uint8  
 14  cmonth_5.0   39455 non-null  uint8  
 15  cmonth_6.0   39455 non-null  uint8  
 16  cmonth_7.0   39455 non-null  uint8  
 17  cmonth_8.0   39455 non-null  uint8  
 18  cmonth_9.0 

# Create Dummies & Numeric Datasets

Form **two** separate datasets:
- DataFrame with **dummy month and hour variables**
- DataFrame with **single numerical month and hour variables**

In [11]:
# The dummy seasonality dataset
df_dum = df.drop(['mth_sin', 'mth_cos', 'hour_sin', 'hour_cos', 'mth_num', 'hour_num'], axis=1)

In [12]:
# The numeric seasonality dataset
df_num = df[['load', 'temp', 'humid', 'target', 'cwkend', 'mth_num', 'hour_num']]

# Preparing Data for Optimization

### Train-Test Split

In [13]:
# Set test set at 20% of data
sample = int(len(df)*0.2)

In [14]:
sample

7891

In [15]:
# Split dummy seasonality dataset
df_dumt = df_dum.iloc[-sample:]
df_dum = df_dum.iloc[:-sample]

In [16]:
df_dum.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31564 entries, 2015-07-02 00:00:00 to 2019-02-06 03:00:00
Data columns (total 39 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   load         31564 non-null  int32  
 1   temp         31564 non-null  float64
 2   humid        31564 non-null  int32  
 3   target       31564 non-null  int32  
 4   cwkend       31564 non-null  uint8  
 5   cmonth_2.0   31564 non-null  uint8  
 6   cmonth_3.0   31564 non-null  uint8  
 7   cmonth_4.0   31564 non-null  uint8  
 8   cmonth_5.0   31564 non-null  uint8  
 9   cmonth_6.0   31564 non-null  uint8  
 10  cmonth_7.0   31564 non-null  uint8  
 11  cmonth_8.0   31564 non-null  uint8  
 12  cmonth_9.0   31564 non-null  uint8  
 13  cmonth_10.0  31564 non-null  uint8  
 14  cmonth_11.0  31564 non-null  uint8  
 15  cmonth_12.0  31564 non-null  uint8  
 16  chour_1.0    31564 non-null  uint8  
 17  chour_2.0    31564 non-null  uint8  
 18  chour_3.0  

In [17]:
df_dumt.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7891 entries, 2019-02-06 04:00:00 to 2019-12-31 22:00:00
Data columns (total 39 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   load         7891 non-null   int32  
 1   temp         7891 non-null   float64
 2   humid        7891 non-null   int32  
 3   target       7891 non-null   int32  
 4   cwkend       7891 non-null   uint8  
 5   cmonth_2.0   7891 non-null   uint8  
 6   cmonth_3.0   7891 non-null   uint8  
 7   cmonth_4.0   7891 non-null   uint8  
 8   cmonth_5.0   7891 non-null   uint8  
 9   cmonth_6.0   7891 non-null   uint8  
 10  cmonth_7.0   7891 non-null   uint8  
 11  cmonth_8.0   7891 non-null   uint8  
 12  cmonth_9.0   7891 non-null   uint8  
 13  cmonth_10.0  7891 non-null   uint8  
 14  cmonth_11.0  7891 non-null   uint8  
 15  cmonth_12.0  7891 non-null   uint8  
 16  chour_1.0    7891 non-null   uint8  
 17  chour_2.0    7891 non-null   uint8  
 18  chour_3.0   

In [18]:
# Target variables
y_dumt = df_dumt.pop('target')
y_dum = df_dum.pop('target')

In [19]:
# Split categorical seasonality dataset
df_numt = df_num.iloc[-sample:]
df_num = df_num.iloc[:-sample]

In [20]:
# Target variables
y_numt = df_numt.pop('target')
y_num = df_num.pop('target')

### Scaling data

In [21]:
# Using MinMaxScaler due to the comparison with the dummy seasonality data
scaler = MinMaxScaler()

In [22]:
# Scaling the dummies dataset
X_dum = scaler.fit_transform(df_dum)
X_dumt = scaler.transform(df_dumt)

In [23]:
# Scaling the categorical dataset
X_num = scaler.fit_transform(df_num)
X_numt = scaler.transform(df_numt)

In [24]:
#Make an inner and outer validation scheme for Nested Cross-Validation
time_split = TimeSeriesSplit(n_splits = 3)

# Randomized Search CV

The first optimization is undertaken on the **dummy seasonality** dataset

In [25]:
# Lasso Regression
lasso = Lasso(random_state=11)

las_dist = {'fit_intercept': [1, 0],
            'alpha': uniform(0.001, 2)}


In [26]:
rs_las = RandomizedSearchCV(lasso, las_dist, cv=time_split, scoring='neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [27]:
rs_las.fit(X_dum, y_dum)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Lasso(random_state=11), n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C855A30>,
                                        'fit_intercept': [1, 0]},
                   scoring='neg_mean_squared_error', verbose=1)

In [28]:
# Ridge Regression
ridge = Ridge(random_state=11)

rdg_dist = {'fit_intercept': [1, 0],
            'solver': ['lsqr', 'sag', 'cholesky'],
            'alpha': uniform(0.001, 2)}


In [29]:
rs_rdg = RandomizedSearchCV(ridge, rdg_dist, cv=time_split, scoring='neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [30]:
rs_rdg.fit(X_dum, y_dum)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Ridge(random_state=11), n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C6E7880>,
                                        'fit_intercept': [1, 0],
                                        'solver': ['lsqr', 'sag', 'cholesky']},
                   scoring='neg_mean_squared_error', verbose=1)

In [31]:
# Random Forest
rf = RandomForestRegressor(random_state=11)

rf_dist = {'n_estimators': randint(50, 500),
           'min_samples_split': randint(2, 9),
           'max_features': ['auto', 'log2', 'sqrt']}


In [32]:
rs_rf = RandomizedSearchCV(rf, rf_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                           n_jobs=-1, verbose=1)

In [33]:
rs_rf.fit(X_dum, y_dum)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=RandomForestRegressor(random_state=11), n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'log2',
                                                         'sqrt'],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C855760>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F8100>},
                   scoring='neg_mean_squared_error', verbose=1)

In [34]:
# XG Boost Regressor
xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=11)

xgb_dist = {'n_estimators': randint(50, 500),
            'subsample': [0.5, 0.7, 1],
            'eta': uniform(0.05, 1.0),
            'gamma': randint(0, 300)}


In [35]:
rs_xgb = RandomizedSearchCV(xgb, xgb_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [36]:
rs_xgb.fit(X_dum, y_dum)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_w...
                                          verbosity=None),
                   n_jobs=-1,
                   param_distributions={'eta': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F8790>,
                                        'gamma': <scipy.stats._

In [37]:
# Support Vector Regressor
svm = SVR()

svm_dist = {'kernel': ['rbf', 'linear', 'poly'],
            'gamma': ['scale', 'auto', 0.2], 
            'epsilon': uniform(0.01, 3),
            'C': uniform(0.1, 5)}


In [38]:
rs_svm = RandomizedSearchCV(svm, svm_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [39]:
rs_svm.fit(X_dum, y_dum)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=SVR(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F0F70>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C2944F0>,
                                        'gamma': ['scale', 'auto', 0.2],
                                        'kernel': ['rbf', 'linear', 'poly']},
                   scoring='neg_mean_squared_error', verbose=1)

In [40]:
# Multi-Layer Perceptron
# Typically 1-2 hidden layers are adequate, and the optimal size of the first hidden layer is usually... 
# between that of the input and the output layers, or 38 and 1 in this case (38 is the max with all the time dummies)

mlp = MLPRegressor(early_stopping=True, max_iter=10000, random_state=11)

mlp_dist = {'hidden_layer_sizes': [(15,), (22,), (30,), (15,8), (22,8), (22,15), (30,8), (30,15)],
            'alpha': uniform(0.01, 5),
            'activation': ['relu', 'tanh'],
            'solver': ['lbfgs', 'adam']}


In [41]:
rs_mlp = RandomizedSearchCV(mlp, mlp_dist, cv=time_split, scoring = 'neg_mean_squared_error', 
                            n_jobs=-1, verbose=1)

In [42]:
rs_mlp.fit(X_dum, y_dum)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\Tan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=MLPRegressor(early_stopping=True, max_iter=10000,
                                          random_state=11),
                   n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F09A0>,
                                        'hidden_layer_sizes': [(15,), (22,),
                                                               (30,), (15, 8),
                                                               (22, 8),
                                                               (22, 15),
                                                               (30, 8),
                                                               (30, 15)],
                                        'solver': ['lbfgs', 'adam']},
                   sco

### Best parameters found

In [43]:
rs_las.best_params_

{'alpha': 0.5750998609261154, 'fit_intercept': 1}

In [44]:
rs_rdg.best_params_

{'alpha': 0.1709174847023479, 'fit_intercept': 1, 'solver': 'sag'}

In [45]:
rs_rf.best_params_

{'max_features': 'auto', 'min_samples_split': 6, 'n_estimators': 171}

In [46]:
rs_xgb.best_params_

{'eta': 0.2844429935096328, 'gamma': 295, 'n_estimators': 461, 'subsample': 1}

In [47]:
rs_svm.best_params_

{'C': 5.071605234485836,
 'epsilon': 0.8856124221777116,
 'gamma': 'scale',
 'kernel': 'poly'}

In [48]:
rs_mlp.best_params_

{'activation': 'relu',
 'alpha': 4.879482142248864,
 'hidden_layer_sizes': (30, 15),
 'solver': 'lbfgs'}

# Cross-Validation & Scoring on Dummies Data

### Cross-validation on training data

In [49]:
las_scores_dum = cross_val_score(rs_las, X_dum, y_dum, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [50]:
rdg_scores_dum = cross_val_score(rs_rdg, X_dum, y_dum, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [51]:
rf_scores_dum = cross_val_score(rs_rf, X_dum, y_dum, cv = time_split, n_jobs=-1, 
                                scoring = 'neg_mean_absolute_error')

In [52]:
xgb_scores_dum = cross_val_score(rs_xgb, X_dum, y_dum, cv = time_split, 
                                 scoring = 'neg_mean_absolute_error')

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [53]:
svm_scores_dum = cross_val_score(rs_svm, X_dum, y_dum, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [54]:
mlp_scores_dum = cross_val_score(rs_mlp, X_dum, y_dum, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [55]:
cv_dict_dum = {
    'Lasso Regression': -np.round(las_scores_dum.mean(), 4),
    'Ridge Regression': -np.round(rdg_scores_dum.mean(), 4),
    'Random Forest': -np.round(rf_scores_dum.mean(), 4),
    'Xtreme Gradient Boost': -np.round(xgb_scores_dum.mean(), 4),
    'Support Vector Machine': -np.round(svm_scores_dum.mean(), 4),
    'Multi-Layer Perceptron': -np.round(mlp_scores_dum.mean(), 4),
}

In [56]:
df_cv_dum = pd.DataFrame({'Model': cv_dict_dum.keys(), 'Average MAE': cv_dict_dum.values()})

### Scoring test data

In [57]:
def reg_scoring(X, y, reg_dict):
    '''
    Objective: Cycles through a dictionary of trained models, using them to make predictions, scores those 
    predictions on MAE, MSE & RMSE, and generates DataFrames of the scores and model predictions respectively
    
    X: DataFrame containing the explanatory variables
    
    y: Target variable
    
    reg_dict: Dictionary of trained/fitted models
    '''
    
    test1_scores = []
    test2_scores = []
    
    df_pred = pd.DataFrame(columns=reg_dict.keys()) # Columns of DF will accord with reg_dict keys
    
    # Loop through Dictionary items
    for key, reg in reg_dict.items():
        
        pred_y = reg.predict(X)
        df_pred[key] = pd.Series(pred_y).transpose()
        
        # Computing test scores for each model
        test1_scores.append(round(mean_absolute_error(y, pred_y), 4))
        test2_scores.append(round(mean_squared_error(y, pred_y, squared=False), 4))
        
    # Generate results DataFrame
    results = pd.DataFrame({'Model': list(reg_dict.keys()), 
                            'Mean Absolute Error': test1_scores,
                            'Root Mean Squared Error': test2_scores
                            })
    
    # Add target variable to the DataFrame of predictions
    df_pred['Target'] = y.tolist()
    
    return results, df_pred


In [58]:
# Dictionary of TRAINED models
reg_dict = {
    'Lasso Regression': rs_las,
    'Ridge Regression': rs_rdg,
    'Random Forest': rs_rf,
    'Xtreme Gradient Boost': rs_xgb,
    'Support Vector Machine': rs_svm,
    'Multi-Layer Perceptron': rs_mlp
}

In [59]:
scores_dum, df_pred_dum = reg_scoring(X_dumt, y_dumt, reg_dict)

# Refit on Numerical Seasonality Data & Scoring

In [60]:
rs_las.fit(X_num, y_num)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Lasso(random_state=11), n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C855A30>,
                                        'fit_intercept': [1, 0]},
                   scoring='neg_mean_squared_error', verbose=1)

In [61]:
rs_rdg.fit(X_num, y_num)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Ridge(random_state=11), n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C6E7880>,
                                        'fit_intercept': [1, 0],
                                        'solver': ['lsqr', 'sag', 'cholesky']},
                   scoring='neg_mean_squared_error', verbose=1)

In [62]:
rs_rf.fit(X_num, y_num)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=RandomForestRegressor(random_state=11), n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'log2',
                                                         'sqrt'],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C855760>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F8100>},
                   scoring='neg_mean_squared_error', verbose=1)

In [63]:
rs_xgb.fit(X_num, y_num)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_w...
                                          verbosity=None),
                   n_jobs=-1,
                   param_distributions={'eta': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F8790>,
                                        'gamma': <scipy.stats._

In [64]:
rs_svm.fit(X_num, y_num)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=SVR(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F0F70>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C2944F0>,
                                        'gamma': ['scale', 'auto', 0.2],
                                        'kernel': ['rbf', 'linear', 'poly']},
                   scoring='neg_mean_squared_error', verbose=1)

In [65]:
rs_mlp.fit(X_num, y_num)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=MLPRegressor(early_stopping=True, max_iter=10000,
                                          random_state=11),
                   n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA8C1F09A0>,
                                        'hidden_layer_sizes': [(15,), (22,),
                                                               (30,), (15, 8),
                                                               (22, 8),
                                                               (22, 15),
                                                               (30, 8),
                                                               (30, 15)],
                                        'solver': ['lbfgs', 'adam']},
                   sco

### Best parameters found

In [66]:
rs_las.best_params_

{'alpha': 0.4265774130674401, 'fit_intercept': 1}

In [67]:
rs_rdg.best_params_

{'alpha': 0.023670267405791923, 'fit_intercept': 1, 'solver': 'sag'}

In [68]:
rs_rf.best_params_

{'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 184}

In [69]:
rs_xgb.best_params_

{'eta': 0.05987395786498813,
 'gamma': 68,
 'n_estimators': 427,
 'subsample': 0.5}

In [70]:
rs_svm.best_params_

{'C': 4.718712691702722,
 'epsilon': 0.2726025400944049,
 'gamma': 'scale',
 'kernel': 'linear'}

In [71]:
rs_mlp.best_params_

{'activation': 'relu',
 'alpha': 2.4111360628432275,
 'hidden_layer_sizes': (30, 15),
 'solver': 'lbfgs'}

# Cross-Validation & Scoring on Categorical Data

In [72]:
las_scores_num = cross_val_score(rs_las, X_num, y_num, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [73]:
rdg_scores_num = cross_val_score(rs_rdg, X_num, y_num, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [74]:
rf_scores_num = cross_val_score(rs_rf, X_num, y_num, cv = time_split, n_jobs=-1, 
                                scoring = 'neg_mean_absolute_error')

In [75]:
xgb_scores_num = cross_val_score(rs_xgb, X_num, y_num, cv = time_split, 
                                 scoring = 'neg_mean_absolute_error')

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [76]:
svm_scores_num = cross_val_score(rs_svm, X_num, y_num, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [77]:
mlp_scores_num = cross_val_score(rs_mlp, X_num, y_num, cv = time_split, n_jobs=-1, 
                                 scoring = 'neg_mean_absolute_error')

In [78]:
cv_dict_num = {
    'Lasso Regression': -np.round(las_scores_num.mean(), 4),
    'Ridge Regression': -np.round(rdg_scores_num.mean(), 4),
    'Random Forest': -np.round(rf_scores_num.mean(), 4),
    'Xtreme Gradient Boost': -np.round(xgb_scores_num.mean(), 4),
    'Support Vector Machine': -np.round(svm_scores_num.mean(), 4),
    'Multi-Layer Perceptron': -np.round(mlp_scores_num.mean(), 4),
}

In [79]:
df_cv_num = pd.DataFrame({'Model': cv_dict_num.keys(), 'Average MAE': cv_dict_num.values()})

### Scoring on test data

In [80]:
scores_num, df_pred_num = reg_scoring(X_numt, y_numt, reg_dict)

In [81]:
df_pred_dum['date'] = df_dumt.index
df_pred_num['date'] = df_numt.index

In [82]:
#df_pred_dum.to_csv(r'C:\\ny_pred_dum.csv', index=False)

In [83]:
#df_pred_num.to_csv(r'C:\\ny_pred_num.csv', index=False)

# Comparative Scores

### Cross-validation scores

In [84]:
df_cv_dum

,Model,Average MAE
0,Lasso Regression,258.2085
1,Ridge Regression,252.8884
2,Random Forest,207.8334
3,Xtreme Gradient Boost,143.0849
4,Support Vector Machine,1189.7579
5,Multi-Layer Perceptron,194.7623


In [85]:
df_cv_num

,Model,Average MAE
0,Lasso Regression,570.5975
1,Ridge Regression,571.5516
2,Random Forest,140.0205
3,Xtreme Gradient Boost,132.8851
4,Support Vector Machine,1557.9531
5,Multi-Layer Perceptron,328.7591


### Test data scores

In [86]:
scores_dum

,Model,Mean Absolute Error,Root Mean Squared Error
0,Lasso Regression,240.4340,309.8406
1,Ridge Regression,240.7296,308.9430
2,Random Forest,177.0530,261.8896
3,Xtreme Gradient Boost,121.2744,162.9995
4,Support Vector Machine,1012.3491,1351.1937
5,Multi-Layer Perceptron,109.0606,146.8422


In [87]:
scores_num

,Model,Mean Absolute Error,Root Mean Squared Error
0,Lasso Regression,524.9513,669.4484
1,Ridge Regression,525.1390,669.4783
2,Random Forest,126.7053,172.8285
3,Xtreme Gradient Boost,114.2283,152.9164
4,Support Vector Machine,895.7971,1143.5224
5,Multi-Layer Perceptron,185.6066,247.6233
